# DAD for Wave by Triplet Loss

## Settings

In [1]:
from pathlib import Path
import sys

In [2]:
import torch
from torchvision import transforms

In [3]:
sys.path.append('../src')

import utils
from dataloader import ESC50DatasetTriplet
from models import EmbeddingNet, TripletNet
from loss_functions import TripletLoss
from trainer import fit

In [4]:
no_cuda = False
seed = 0

In [5]:
data_dir = Path().cwd().parent/'data'
raw_data_dir = data_dir/'external'/'ESC-50'
processed_data_dir = data_dir/'processed'/'ESC-50'
models_dir = Path().cwd().parent/'models'

In [6]:
model_path_tmp = models_dir/'model_tmp.pth'
model_path = models_dir/'model.pth'
embeddingnet_path = models_dir/'embeddingnet.pth'

In [7]:
metadata_path = raw_data_dir/'meta'/'esc50.csv'
audio_dir = raw_data_dir/'audio'
spectrogram_dir = processed_data_dir/'spectrogram'

In [8]:
train_metadata_path = processed_data_dir/'metadata_train.csv'
valid_metadata_path = processed_data_dir/'metadata_valid.csv'
test_metadata_path = processed_data_dir/'metadata_test.csv'

In [9]:
use_cuda = torch.cuda.is_available() and (not no_cuda)
device = 'cuda' if use_cuda else 'cpu'
torch.manual_seed(seed)

print('device: {}'.format(device))

device: cuda


## Train

### Hyperparameters

In [10]:
input_size = (257, 431)
output_size = 128

margin = 0.8

lr = 5e-6
weight_decay = 1e-5

batch_size = 2
n_epochs = 30

log_interval = 100

### Dataloaders

In [11]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    ])

In [13]:
dataset_train = ESC50DatasetTriplet(
    train_metadata_path, audio_dir, spectrogram_dir, transform,
    )

In [14]:
dataset_valid = ESC50DatasetTriplet(
    valid_metadata_path, audio_dir, spectrogram_dir, transform,
    )

In [15]:
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, **kwargs)

In [16]:
dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, **kwargs)

### Model

In [17]:
embedding_net = EmbeddingNet(input_size, output_size).to(device)
model = TripletNet(embedding_net).to(device)

In [18]:
loss_function = TripletLoss(margin)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

### Train

In [19]:
torch.backends.cudnn.benchmark = True

In [20]:
fit(
    dataloader_train,
    dataloader_valid,
    model, loss_function, optimizer, scheduler,
    n_epochs, use_cuda, log_interval,
    model_path=model_path_tmp,
    )

Train: [0/1600 (0%)]	Loss: 0.793592
Train: [200/1600 (12%)]	Loss: 0.796300
Train: [400/1600 (25%)]	Loss: 0.794598
Train: [600/1600 (38%)]	Loss: 0.787550
Train: [800/1600 (50%)]	Loss: 0.746206
Train: [1000/1600 (62%)]	Loss: 0.683375
Train: [1200/1600 (75%)]	Loss: 0.653792
Train: [1400/1600 (88%)]	Loss: 0.620712
Epoch: 1/30. Train set: Average loss: 0.7199
Epoch: 1/30. Validation set: Average loss: 0.5594
Train: [0/1600 (0%)]	Loss: 0.732125
Train: [200/1600 (12%)]	Loss: 0.689834
Train: [400/1600 (25%)]	Loss: 0.633253
Train: [600/1600 (38%)]	Loss: 0.694788
Train: [800/1600 (50%)]	Loss: 0.637754
Train: [1000/1600 (62%)]	Loss: 0.611609
Train: [1200/1600 (75%)]	Loss: 0.606764
Train: [1400/1600 (88%)]	Loss: 0.678660
Epoch: 2/30. Train set: Average loss: 0.6489
Epoch: 2/30. Validation set: Average loss: 0.5460
Train: [0/1600 (0%)]	Loss: 0.346349
Train: [200/1600 (12%)]	Loss: 0.664487
Train: [400/1600 (25%)]	Loss: 0.633415
Train: [600/1600 (38%)]	Loss: 0.635110
Train: [800/1600 (50%)]	Loss: 0.6

Train: [200/1600 (12%)]	Loss: 0.494419
Train: [400/1600 (25%)]	Loss: 0.524357
Train: [600/1600 (38%)]	Loss: 0.597911
Train: [800/1600 (50%)]	Loss: 0.555895
Train: [1000/1600 (62%)]	Loss: 0.548066
Train: [1200/1600 (75%)]	Loss: 0.552301
Train: [1400/1600 (88%)]	Loss: 0.558375
Epoch: 21/30. Train set: Average loss: 0.5433
Epoch: 21/30. Validation set: Average loss: 0.4644
Train: [0/1600 (0%)]	Loss: 0.000000
Train: [200/1600 (12%)]	Loss: 0.566640
Train: [400/1600 (25%)]	Loss: 0.536603
Train: [600/1600 (38%)]	Loss: 0.493391
Train: [800/1600 (50%)]	Loss: 0.494045
Train: [1000/1600 (62%)]	Loss: 0.557401
Train: [1200/1600 (75%)]	Loss: 0.630864
Train: [1400/1600 (88%)]	Loss: 0.609256
Epoch: 22/30. Train set: Average loss: 0.5449
Epoch: 22/30. Validation set: Average loss: 0.4733
Train: [0/1600 (0%)]	Loss: 1.140111
Train: [200/1600 (12%)]	Loss: 0.507648
Train: [400/1600 (25%)]	Loss: 0.630149
Train: [600/1600 (38%)]	Loss: 0.549085
Train: [800/1600 (50%)]	Loss: 0.482042
Train: [1000/1600 (62%)]	L

In [21]:
# save best model as other name
model.load_state_dict(torch.load(str(model_path_tmp)))

torch.save(model.state_dict(), str(model_path))
torch.save(embedding_net.state_dict(), str(embeddingnet_path))